In [ ]:
import cobra

In [ ]:
model = cobra.io.read_sbml_model('AH_model.xml')

In [ ]:
class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [ ]:
prod = []
reac = []
for reaction in model.reactions:
    if '<=>' in reaction.reaction:
        reaction = reaction.reaction.split('<=>')
        reac.append(list(map(lambda x: x[-5:],str(reaction[0]).replace(' ','').split('+'))))
        prod.append(list(map(lambda x: x[-5:],str(reaction[1]).replace(' ','').split('+'))))
        reac.append(list(map(lambda x: x[-5:],str(reaction[1]).replace(' ','').split('+'))))
        prod.append(list(map(lambda x: x[-5:],str(reaction[0]).replace(' ','').split('+'))))
    elif '-->' in reaction.reaction:
        reaction = reaction.reaction.split('-->')
        reac.append(list(map(lambda x: x[-5:],str(reaction[0]).replace(' ','').split('+'))))
        prod.append(list(map(lambda x: x[-5:],str(reaction[1]).replace(' ','').split('+'))))
reac = list(map(lambda x: list(set(x)),reac))
prod = list(map(lambda x: list(set(x)),prod))
graph = {}
cont = 0
for i in reac:
    cont+=1
    for j in i:
        if j in graph.keys():
            graph[j].append(prod[cont-1])
        else:
            graph[j] = []
            for h in prod[cont-1]:
                graph[j].append(h)
for i in graph.keys():
    if type(graph[i]) == str:
        new = [graph[i]]
    else:
        new = []
        for j in graph[i]:
            if type(j) !=str:
                new+=j
            else:
                new.append(j)
    graph[i] = new

In [ ]:
edges = []
block = ['C00015','C00080','C00001',
         'C00003','C00004','C00001',
         'C00002','C00105','C00075',
         'C00004','C00005','C00006',
         'C00003']#add metabolites (kegg id) that cant count as step for path eg. ATP
big_w = []
for i in block:
    big_w = big_w+list(find_metabolite(i).keys())[1:]
for start in graph:
    if start == '':
        continue
    for end in graph[start]:
        if end == '':
            continue
        #add weight
        if start in big_w:
            vertex = (start,end,1000)
        elif end in big_w:
            vertex = (start,end,1000)
        else:
            vertex = (start,end,1)
        edges.append(vertex)

In [ ]:
graph = Graph()
for edge in edges:
    graph.add_edge(*edge)

# Algoritmo de dijsktra

In [ ]:
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    #return path
    #returning the path with the names instead of ids
    ids =list(map(lambda x: model.metabolites.get_by_id(x).name, path))
    return dict(zip(path, ids))

In [ ]:
#dijsktra(graph,reagente,Produto)